In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.basics import *
from fastai.callback.all import *
from fastai.vision.all import *
from fastai.medical.imaging import *

In [ ]:
cassava_path = Path('/kaggle/input/cassava-leaf-disease-classification')
cassava_path

In [ ]:
train_csv = pd.read_csv(cassava_path/'train.csv')
imgs = get_image_files(cassava_path/'train_images')
imgs,train_csv.head()

In [ ]:
def gety(path):
    name = path.name
    return train_csv[train_csv['image_id'] == name].label.values[0]
# def get_x(row): return cassava_path/'train_images'/ row.image_id
# def get_y(row): return row.label

In [ ]:
dblock = DataBlock(blocks    = (ImageBlock, CategoryBlock),
                   get_items = get_image_files,
                   get_y = gety,
                  splitter = RandomSplitter(seed = 42),
                  item_tfms = Resize(460),
                  batch_tfms = [*aug_transforms(size = 224,min_scale = 0.75),Normalize.from_stats(*imagenet_stats)]
                  )

In [ ]:
ds = dblock.dataloaders(cassava_path/'train_images')

In [ ]:
ds.show_batch()

In [ ]:
learner = cnn_learner(ds,resnet34,metrics=[error_rate,accuracy])

In [ ]:
learner.lr_find()

In [ ]:
learner.fine_tune(5,base_lr = 1e-2)

In [ ]:
interp = ClassificationInterpretation.from_learner(learner)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
learner.recorder.plot_loss()

In [ ]:
sample_df = pd.read_csv(cassava_path/'sample_submission.csv')
sample_df.head()
sample_copy = sample_df.copy()
sample_copy['image_id'] = sample_copy['image_id'].apply(lambda x: f'test_images/{x}')
sample_copy.head()

In [ ]:
test_items = get_image_files(cassava_path/'test_images')
sample_df = pd.read_csv(cassava_path/'sample_submission.csv')
# sample_df.head()
# sample_copy = sample_df.copy()
# sample_copy['image_id'] = sample_copy['image_id'].apply(lambda x: f'test_images/{x}')
# sample_copy.head()

In [ ]:
test_dl = learner.dls.test_dl(test_items)

In [ ]:
preds, _ = learner.tta(dl=test_dl)
preds
sample_df['label'] = preds.argmax(dim=-1).numpy()
sample_df.to_csv('/kaggle/working/submission.csv',index=False)